In [1]:
# Importing important library
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

In [2]:
#URL to exract news
url = 'https://www.aajtak.in/uttar-pradesh'

In [65]:
#Importing selenium libraries for automation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException


# Function to extract data from a single page
def extract_data_from_page(soup):
    h2_tags = soup.find_all('h2')
    data_list = []
    for h2_tag in h2_tags:
        anchor_tag = h2_tag.a
        if anchor_tag:
            title = anchor_tag.text
            link = anchor_tag['href']
            data_dict = {
                "title": title,
                "link": link
            }
            data_list.append(data_dict)
    return data_list

# Function to click on the "Load More" button and wait for new content to load
def click_load_more_button(driver):
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "load_more"))
        )
        load_more_button.click()
        # Wait for new content to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h2 a"))
        )
        return True
    except StaleElementReferenceException:
        return False
    except TimeoutException:
        return False

# Set up the WebDriver (you can use other browser drivers if desired)
driver = webdriver.Chrome()

driver.get(url)

# Extract data from the initial page
page = driver.page_source
soup = BeautifulSoup(page, 'lxml')
data_list = extract_data_from_page(soup)

# Loop to click "Load More" and extract additional data until no more content is loaded
while click_load_more_button(driver):
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    new_data_list = extract_data_from_page(soup)
    data_list.extend(new_data_list)

# Close the WebDriver
driver.quit()

In [71]:
# Creating dataframe from extracted text
link_data = pd.DataFrame(data_list)

In [72]:
# Function for getting clean hindi text 
import re

def extract_hindi_text(input_text):
    # Define the regex pattern to match Hindi characters (Devanagari script)
    hindi_pattern = r'[\u0900-\u097F]+'
    
    # Use re.findall to find all occurrences of the pattern in the input text
    hindi_text_list = re.findall(hindi_pattern, input_text)
    
    # Join the list of matches into a single string
    hindi_text = ' '.join(hindi_text_list)
    
    return hindi_text

In [73]:
text = []
for news in link_data['link']:
    news_page = r.get(news).text
    data = bs(news_page,'lxml')
    para = data.find_all('p')
    for i in para:
        clean = extract_hindi_text(i.text)
        text.append(clean)